<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego 
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej" 
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

# Linear regression 

In this exercise you will use linear regression to predict flat prices. Training will be handled via gradient descent and we will:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a non-standard loss function.

Let's start with getting the data.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2022-11-10 16:33:17--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.71.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.71.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zey0gx91pna8irj/mieszkania.csv [following]
--2022-11-10 16:33:18--  https://www.dropbox.com/s/dl/zey0gx91pna8irj/mieszkania.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb7746e2dd0d81c10e4ba99eeda.dl.dropboxusercontent.com/cd/0/get/BwcyLp1Ae2JkIudXyBGAVkEdYpksjcELzM8CiMTlofzpdD5-kcQQGEufLEORz1XdDIWbfkq1LTE2EqapsHv2JZ6co9wz8pRROcSBBdl2OCyzKs-vmd_b5ZJCWuDt2jC2WUvcE4ruIFpF5IVe9OccuDXZsVtXyebQmpcYEU6ZYCUnxw/file?dl=1# [following]
--2022-11-10 16:33:18--  https://ucb7746e2dd0d81c10e4ba99eeda.dl.dropboxusercontent.com/cd/0/get/BwcyLp1Ae2JkIudXyBGAVkEdYpksjcELzM8CiMTlofzpdD5-kcQQGEufLEORz1XdDIWbf

In [2]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilosc_sypialni,ilosc_lazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

def plot(title, losses):
    df = pd.concat(losses)
    fig = px.line(df, log_y=True, title=title, template='plotly_dark')
    fig.show()

def prepare(df):
    labels = df['cena'].to_numpy()
    df.drop('cena', axis=1, inplace=True)
    df = df.join(pd.get_dummies(df['dzielnica']))
    df.drop('dzielnica', axis=1, inplace=True)
    return df, labels

data, prices = prepare(pd.read_csv('mieszkania.csv'))

Let's start with implementing the loss function.

In [4]:
def msle(ys, ps):
    assert len(ys) == len(ps)
    t = np.log(1 + ys) - np.log(1 + ps)
    return np.mean(t ** 2)

def mse_log(ys, ps):
    return mse(np.exp(ys) - 1, np.exp(ps) - 1)

def mse(ys, ps):
    return np.mean((ys - ps) ** 2)
    

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [5]:
###################################################
# Compute msle for outputing the mean price #
###################################################

def msle_constant(ys, constant):
  return msle(ys, np.full_like(ys, constant))

msle_constant(prices, 1000.)

37.162711400302044

Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [6]:
#############################################
# Find this constant and compute msle #
#############################################
msle_constant(prices, np.mean(prices)), msle_constant(prices, np.exp(np.mean(np.log(prices))))

(0.3915250389336738, 0.36488961221465716)

Now, let's implement a standard linear regression model. 

In [7]:
##########################################################
# Implement linear regression and compute its msle #
##########################################################

def normalize(df):
   # Data normalization
    data_copy = df.copy(deep=True)
    def normalize_field(field):
      t = data_copy[field]
      data_copy[field] = (t - np.mean(t)) / np.std(t)
    
    [normalize_field(field) for field in data_copy.head()]
  
   
    # Bias trick with column filled with ones
    X = data_copy.to_numpy()
    ones = np.full(X.shape[0], 1).reshape(-1, 1)
    X = np.concatenate((ones, X), axis=1)

    return X

def linear_regression(description, data, labels, lr, epoch_count, loss_fun, opt_msle=False, ret_pred=False):
    X = normalize(data)

    # Weights initialization
    w = np.random.rand(X.shape[1])
    w[0] = np.mean(labels)

    def predict(X):
        return np.einsum("i,ji->j", w, X)

    losses = []
    for i in range(epoch_count):
        predictions = predict(X)
        losses.append(loss_fun(labels, predictions))

        if not opt_msle:
          gradient = 2 * np.mean((predictions - labels).reshape(-1, 1) * X, axis=0)
        else:
          gradient = 2 * ((np.log(1 + predictions) - np.log(1 + labels))/(1 + predictions) @ X) / len(labels)
        w = w - lr * gradient

    #display(pd.DataFrame(data={'label': labels, 'prediction': predict(X)})[:10])
    if ret_pred:
      return predict
    else:
      return pd.DataFrame(losses, columns=[description])

In [15]:
plot("aaa", [linear_regression("msle1", data, prices, 0.01, 1000, msle)])

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint: 
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change the algorithm.
</sup></sub></sup></sub></sup></sub>

In [9]:
#############################################
# Optimize msle and compare the error #
#############################################

pred_2 = linear_regression("log pred", data, np.log(1 + prices), 0.01, 2000, mse)

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint: 
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

In [10]:
# It is not enough
# Dodac logarytm z nowych kolumn
districts = ['grodziskowo', 'mokotowo', 'ochotowo', 'wolowo']
t = data[districts].multiply(data['m2'], axis='index')
data_feat = data.copy(deep=True)
data_feat = data_feat.join(t.add_suffix('_area'))
data_feat.drop(columns=districts, inplace=True)
display(data_feat)
plot("Area times district" ,[
  linear_regression("bad log prediction", data_feat, np.log(1 + prices), 0.01, 2000, mse),
  linear_regression("", data_feat, prices, 0.01, 2000, msle)
])

,m2,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,grodziskowo_area,mokotowo_area,ochotowo_area,wolowo_area
0,104,2,2,1940,1,0,104,0,0
1,43,1,1,1970,1,0,0,43,0
2,128,3,2,1916,1,128,0,0,0
3,112,3,2,1920,1,0,112,0,0
4,149,3,3,1977,0,0,149,0,0
...,...,...,...,...,...,...,...,...,...
195,54,1,1,1992,1,0,54,0,0
196,35,1,1,1985,1,35,0,0,0
197,107,2,2,1935,0,107,0,0,0
198,117,3,2,1978,1,117,0,0,0


In [11]:
def extend_data(dt):
    data = dt.copy()
    data['cena_za_metr'] = data['cena'] / data['m2']
    average_prise = data[['dzielnica', 'cena_za_metr']].groupby(['dzielnica']).mean()
    average_prise.loc['grodziskowo']['cena_za_metr']
    data['srednia_cena_za_metr'] = average_prise.loc[data['dzielnica']].reset_index()['cena_za_metr']
    data['srednia_cena'] = (data['srednia_cena_za_metr'] * data['m2']).astype(int)
    return data.drop(columns=['cena_za_metr'])

xxx = extend_data(pd.read_csv('mieszkania.csv'))
print(msle(prices, xxx['srednia_cena']))
without_log = data_feat.join(xxx['srednia_cena'])
with_log = data_feat.join(np.log(xxx['srednia_cena']))
plot("Feature engineering with average_price, comparison of conversion between normal values and logs", [
    linear_regression("norm->log", without_log, np.log(1 + prices), 0.01, 2000, mse),
    linear_regression("norm->norm", without_log, prices, 0.01, 2000, msle),
    linear_regression("log->log", with_log, np.log(1 + prices), 0.01, 2000, mse),
    linear_regression("log->norm", with_log, prices, 0.01, 2000, msle)
])

0.009148047063425957


In [12]:
# When computing gradient of MSLE
plot("When computing gradient of MSLE", [
     linear_regression("big lr", data_feat, prices, 1000000000, 2000, msle, True)
])

Hint 2: 
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [13]:
###############################################
# Implement the feature engineering part #
###############################################
def multiply_all(df):
    res = pd.concat([df.mul(col[1], axis="index") for col in df.iteritems()], axis=1)
    res.columns = ["_".join([i, j]) for j in df.columns for i in df.columns]
    return res.loc[:, res.any()]

data_feat_mul = multiply_all(data)
display(data_feat_mul)
plot("Multiplied all vs all", [
    linear_regression("lr=0.01", data_feat_mul, prices, 0.01, 2000, msle),
    linear_regression("lr=0.02", data_feat_mul, prices, 0.02, 2000, msle),
    linear_regression("lr=0.05", data_feat_mul, prices, 0.05, 2000, msle)
])

,m2_m2,ilość_sypialni_m2,ilość_łazienek_m2,rok_budowy_m2,parking_podziemny_m2,grodziskowo_m2,mokotowo_m2,ochotowo_m2,wolowo_m2,m2_ilość_sypialni,...,ilość_łazienek_ochotowo,rok_budowy_ochotowo,parking_podziemny_ochotowo,ochotowo_ochotowo,m2_wolowo,ilość_sypialni_wolowo,ilość_łazienek_wolowo,rok_budowy_wolowo,parking_podziemny_wolowo,wolowo_wolowo
0,10816,208,208,201760,104,0,104,0,0,208,...,0,0,0,0,0,0,0,0,0,0
1,1849,43,43,84710,43,0,0,43,0,43,...,1,1970,1,1,0,0,0,0,0,0
2,16384,384,256,245248,128,128,0,0,0,384,...,0,0,0,0,0,0,0,0,0,0
3,12544,336,224,215040,112,0,112,0,0,336,...,0,0,0,0,0,0,0,0,0,0
4,22201,447,447,294573,0,0,149,0,0,447,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2916,54,54,107568,54,0,54,0,0,54,...,0,0,0,0,0,0,0,0,0,0
196,1225,35,35,69475,35,35,0,0,0,35,...,0,0,0,0,0,0,0,0,0,0
197,11449,214,214,207045,0,107,0,0,0,214,...,0,0,0,0,0,0,0,0,0,0
198,13689,351,234,231426,117,117,0,0,0,351,...,0,0,0,0,0,0,0,0,0,0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning:

invalid value encountered in log



In [14]:
##############################################################
#  Test your solution on the training and test datasets #
##############################################################

test_data, test_prices = prepare(pd.read_csv('mieszkania_test.csv'))
test_data_feat_mul = multiply_all(test_data)
best_pred = linear_regression("lr=0.03", data_feat_mul, prices, 0.02, 1000, msle, ret_pred=True)
print("On training dataset: ", msle(prices, best_pred(normalize(data_feat_mul))))
print("On test dataset: ", msle(test_prices, best_pred(normalize(test_data_feat_mul))))

On training dataset:  0.006249116925796507
On test dataset:  0.026392106677483203
